The experiment is done with a random split so we should not detect a data drift

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import xgboost as xgb
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from src.drift import ModelDriftExplainer

# Breast Cancer Data

In [ ]:
dataset = datasets.load_breast_cancer()

In [ ]:
X = pd.DataFrame(dataset.data, columns = dataset.feature_names)
y = dataset.target

In [ ]:
X.head()

# Build XGBoost model

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=2021)

In [ ]:
clf = XGBClassifier(n_estimators=1000,
                    booster="gbtree",
                    objective="binary:logistic",
                    learning_rate=0.05,
                    max_depth=6,
                    use_label_encoder=False,
                    seed=2021)

In [ ]:
clf.fit(X=X_train, y=y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=20, verbose=10)

# Analyze data drift

In [ ]:
drift_explainer = ModelDriftExplainer(clf)

In [ ]:
drift_explainer.fit(X1=X_train, X2=X_valid, y1=y_train, y2=y_valid)

In [ ]:
drift_explainer.plot_prediction_drift()
drift_explainer.get_prediction_drift()

In [ ]:
drift_explainer.plot_prediction_drift(prediction_type='proba')
drift_explainer.get_prediction_drift(prediction_type='proba')

In [ ]:
drift_explainer.plot_tree_based_drift_values(type='node_size')

In [ ]:
drift_explainer.get_tree_based_drift_values(type='node_size')

In [ ]:
drift_explainer.plot_tree_based_drift_values(type='mean_norm')

In [ ]:
drift_explainer.get_tree_based_drift_values(type='mean_norm')

In [ ]:
drift_explainer.plot_target_drift()
drift_explainer.get_target_drift()

In [ ]:
drift_explainer.plot_feature_drift(0)
drift_explainer.get_feature_drift(0)

In [ ]:
drift_explainer.get_feature_drifts()

# With base XGBoost API

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)

In [ ]:
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_valid, y_valid)

In [ ]:
param = {'max_depth': 6,
         'learning_rate': 0.05,
         'objective': 'binary:logistic',
         'booster': 'gbtree',
         'seed': 2021
        }

In [ ]:
bst = xgb.train(param, dtrain, num_boost_round=1000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=20, verbose_eval=10)

In [ ]:
drift_explainer = ModelDriftExplainer(bst)
drift_explainer.fit(X1=X_train, X2=X_valid, y1=y_train, y2=y_valid)